# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: `Ярошенко Алексей Леонидович`

Группа: `MADE-DS-32`

In [377]:
import json

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
import mapclassify
import contextily as ctx
from shapely import geometry
from shapely.geometry import Point, LineString, MultiPolygon, Polygon
import folium
from folium import plugins
from shapely import wkt

from collections import OrderedDict
from OSMPythonTools.data import Data, dictRangeYears, ALL
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
from OSMPythonTools.nominatim import Nominatim

from openrouteservice import client

import rtree
import countryinfo

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

In [156]:
def to_radians(coord):
    return coord * np.pi / 180

def get_haversine(lat1, lon1, lat2, lon2):
    earth_radius = 6_371_000
    lat1, lon1, lat2, lon2 = map(to_radians, (lat1, lon1, lat2, lon2))
    
    inner_part = np.sin((lat2 - lat1)/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2 - lon1) / 2)**2
    unit_distance = 2 * np.arcsin(np.sqrt(inner_part))
    distance = unit_distance * earth_radius
    
    return distance

In [105]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
target = world[world['name'] == 'Afghanistan']['geometry'].iloc[0]
x_center, y_center = target.centroid.coords[0]

x_bounds = [-180., 180.]
y_bounds = [-90., 90.]

history = []
history_bounds = []


# накидываем случайно точку, делим по точке карту на 4 прямоугольгтка и смотрим, в какой попали
for i in range(30):
    
    history_bounds.append((
        x_bounds[:],
        y_bounds[:]
    ))

    x = np.random.uniform(*x_bounds)
    y = np.random.uniform(*y_bounds)
    
    point_afg = Point(x, y)
    
    history.append(point_afg)
    
    if point_afg.within(target):
        print(f'N_iters={i}, {point_afg}')
        break
        
    if x > x_center:
        x_bounds[1] = x
    else:
        x_bounds[0] = x
        
    if y > y_center:
        y_bounds[1] = y
    else:
        y_bounds[0] = y

N_iters=10, POINT (61.8678383639707 33.17134310559288)


Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

In [106]:
m = folium.Map()


points = []
for i, point in enumerate(history):
    points.append({
        'coordinates': [point.coords[0][0], point.coords[0][1]], 
        'time': f'2020-11-{(i+1):02d}T00:00:00',
    })
    
rects = []
for i, (x_bounds, y_bounds) in enumerate(history_bounds):
    rects.append({
        'coordinates': [[[
            [x_bounds[0], y_bounds[0]],
            [x_bounds[1], y_bounds[0]],
            [x_bounds[1], y_bounds[1]],
            [x_bounds[0], y_bounds[1]],
        ]]],
        'time': f'2020-11-{(i+1):02d}T00:00:00',
    })
    
features = []

for point in points:
    features.append({
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': point['coordinates'],
        },
        'properties': {
            'time': point['time'],
            'icon': 'info-sign',
            'color': 'red'
        }
    })
    
for rect in rects:
    features.append({
        'type': 'Feature',
        'geometry': {
            'type': 'MultiPolygon',
            'coordinates': rect['coordinates'],
        },
        'properties': {
            'time': rect['time'],
            'style': {
                'color': 'red',
                'weight': 1,
            },
        }
    })
    
    


plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    period='P1D',
    add_last_point=True,
    auto_play=True,
    loop=True,
    max_speed=1,
    loop_button=True,
    date_options='YYYY/MM/DD',
    time_slider_drag_update=True,
    duration='P2M'
).add_to(m)
    
m

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [11]:
# put your code here
nominatim = Nominatim()
overpass = Overpass()

In [59]:
countries_resp = overpass.query('relation["admin_level"="2"][boundary=administrative];out;').toJSON()['elements']

## 2.1 Расстояние от точки до 5 ближайших * банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости

In [60]:
selected_countries = []

for i, country in enumerate(countries_resp):
    name = country['tags']['name:en']
    country_id = nominatim.query(name).areaId()
    if country_id and country_id % 10 == 0:
        selected_countries.append((country_id, name))

Жилые дома доставал по этой документации: https://wiki.openstreetmap.org/wiki/Key:building

Брал все Accommodation.

In [70]:
country_buildings = []
for (areaId, country) in selected_countries:

    query = overpassQueryBuilder(
        area=areaId, 
        elementType=['node','way','relation'], 
        selector='"building"~"apartments|bungalow|cabin|detached|dormitory|farm|ger|hotel|house|houseboat|residential|semidetached_house|static_caravan|terrace"',
        out='count'
    )

    buildings_response = overpass.query(query, timeout=240)
    buildings = int(buildings_response.toJSON()['elements'][0]['tags']['total'])
    
    country_buildings.append((areaId, country, buildings))

In [71]:
sorted(country_buildings, key=lambda x: x[2], reverse=True)

[(3600059470, 'Brazil', 768727),
 (3600080500, 'Australia', 593526),
 (3602978650, 'Norway', 331485),
 (3600195270, 'Tanzania', 314965),
 (3600295480, 'Portugal', 282526),
 (3600195290, 'Malawi', 97771),
 (3600192780, 'Liberia', 54907),
 (3600184640, 'Bangladesh', 51056),
 (3600196240, 'Uzbekistan', 43094),
 (3600287670, 'Honduras', 33135),
 (3602088990, 'Kosovo', 24310),
 (3600364110, 'Azerbaijan', 22227),
 (3600192830, 'Cameroon', 22140),
 (3602103120, 'Brunei', 15178),
 (3600184840, 'Syria', 13839),
 (3600192800, 'Ethiopia', 11420),
 (3600079510, 'Estonia', 10687),
 (3600535790, 'Comoros', 8286),
 (3600192790, 'Central African Republic', 7915),
 (3600088210, 'Eswatini', 7316),
 (3600536900, 'Antigua and Barbuda', 443),
 (3600535880, 'São Tomé and Príncipe', 112),
 (3600036990, 'Monaco', 56),
 (3602186600, 'Tokelau', 1)]

In [75]:
brasil_area_id = 3600059470

query = overpassQueryBuilder(
    area=brasil_area_id, 
    elementType=['node','way','relation'], 
    selector='"amenity"="atm"',
    out='meta'
)

atms = overpass.query(query, timeout=240)

[overpass] downloading data: [timeout:240][out:json];area(3600059470)->.searchArea;(node["amenity"="atm"](area.searchArea);way["amenity"="atm"](area.searchArea);relation["amenity"="atm"](area.searchArea);); out meta;


In [157]:
rtree_atm = rtree.index.Index()
m = folium.Map()

atm_distance = 0

point_afg_lon, point_afg_lat = point_afg.coords[0][0], point_afg.coords[0][1]

for i, atm in enumerate(atms.toJSON()['elements']):
    if 'lon' in atm and 'lat' in atm:
        rtree_atm.add(i, (atm['lon'], atm['lat'], atm['lon'], atm['lat']))

nearest_atms = rtree_atm.nearest(
    (point_afg_lon, point_afg_lat, point_afg_lon, point_afg_lat),
    num_results=5,
)

for atm_idx in nearest_atms:
    atm = atms.toJSON()['elements'][atm_idx]
    print('ATM:', atm['id'], [atm['lat'], atm['lon']])
    folium.Marker([atm['lat'], atm['lon']]).add_to(m)
    atm_distance += get_haversine(point_afg_lat, point_afg_lon, atm['lat'], atm['lon'])
    
print('Distance sum:', atm_distance)
    

folium.Marker([point_afg.coords[0][1], point_afg.coords[0][0]], icon=folium.Icon(color='green')).add_to(m)  
m.location = [atm['lat'], atm['lon']]
m

ATM: 2600419328 [-3.8406011, -32.4109438]
ATM: 2600419329 [-3.8414331, -32.4109961]
ATM: 2601191548 [-3.8493531, -32.4258847]
ATM: 2600419340 [-3.8565028, -32.4285865]
ATM: 2600419341 [-3.8565364, -32.4285886]
Distance sum: 53202409.29847178


Попали в банкоматы на бразильском острове, ближайшем к Афганистану. Вроде, ок :)

## 2.2 Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице

In [175]:
query = overpassQueryBuilder(
        area=brasil_area_id, 
        elementType='relation', 
        selector=['"admin_level"="6"', '"type"="boundary"', '"boundary"="administrative"'],
        out='meta'
    )

print(query)
# capital = overpass.query(query, timeout=240)

area(3600059470)->.searchArea;(relation["admin_level"="6"]["type"="boundary"]["boundary"="administrative"](area.searchArea);); out meta;


In [218]:
selected_capitals = []

for (country_id, country_name) in selected_countries:
    try:
        capital = countryinfo.CountryInfo(country_name).capital()
        if country_name == 'Australia':
            capital = 'Australian Capital Territory'
            
        if country_name == 'Brazil':
            capital = 'Federal District'
        area_id = nominatim.query(f'{capital}').areaId()
        if area_id is None:
            area_id = nominatim.query(f'{capital}, {country_name}').areaId()
        if area_id:
            selected_capitals.append((area_id, country_name, capital))
        
        
    except Exception as e:
        pass
    
selected_capitals

[nominatim] downloading data: search


[(3600421151, 'Brazil', 'Federal District'),
 (3602164745, 'Estonia', 'Tallinn'),
 (3602354197, 'Australia', 'Australian Capital Territory'),
 (3610249594, 'Bangladesh', 'Dhaka'),
 (3600186228, 'Syria', 'Damascus'),
 (3609319970, 'Liberia', 'Monrovia'),
 (3607322726, 'Central African Republic', 'Bangui'),
 (3601707699, 'Ethiopia', 'Addis Ababa'),
 (3602746229, 'Cameroon', 'Yaoundé'),
 (3601600774, 'Tanzania', 'Dodoma'),
 (3607345524, 'Malawi', 'Lilongwe'),
 (3602216724, 'Uzbekistan', 'Tashkent'),
 (3603850824, 'Honduras', 'Tegucigalpa'),
 (3605400890, 'Portugal', 'Lisbon'),
 (3603764541, 'Azerbaijan', 'Baku'),
 (3600198859, 'Comoros', 'Moroni'),
 (3609377730, 'São Tomé and Príncipe', 'São Tomé'),
 (3602805426, 'Antigua and Barbuda', "Saint John's"),
 (3600036990, 'Monaco', 'Monaco'),
 (3608748136, 'Brunei', 'Bandar Seri Begawan'),
 (3600406091, 'Norway', 'Oslo')]

In [222]:
capital_pharmacies = []
for (areaId, country, capital) in selected_capitals:
    if areaId is None:
        continue

    query = overpassQueryBuilder(
        area=areaId, 
        elementType=['node','way','relation'], 
        selector='"amenity"="pharmacy"',
        out='count'
    )

    pharmacy_response = overpass.query(query, timeout=240)
    pharmacies = int(pharmacy_response.toJSON()['elements'][0]['tags']['total'])
    
    capital_pharmacies.append((areaId, country, capital, pharmacies))

In [221]:
sorted(capital_pharmacies, key=lambda x: x[3], reverse=True)

[(3602216724, 'Uzbekistan', 'Tashkent', 853),
 (3600421151, 'Brazil', 'Federal District', 376),
 (3605400890, 'Portugal', 'Lisbon', 295),
 (3603764541, 'Azerbaijan', 'Baku', 243),
 (3601707699, 'Ethiopia', 'Addis Ababa', 164),
 (3603850824, 'Honduras', 'Tegucigalpa', 161),
 (3602746229, 'Cameroon', 'Yaoundé', 148),
 (3600406091, 'Norway', 'Oslo', 129),
 (3602164745, 'Estonia', 'Tallinn', 82),
 (3602354197, 'Australia', 'Australian Capital Territory', 46),
 (3601600774, 'Tanzania', 'Dodoma', 20),
 (3602805426, 'Antigua and Barbuda', "Saint John's", 14),
 (3600036990, 'Monaco', 'Monaco', 12),
 (3607345524, 'Malawi', 'Lilongwe', 10),
 (3608748136, 'Brunei', 'Bandar Seri Begawan', 4),
 (3609319970, 'Liberia', 'Monrovia', 3),
 (3607322726, 'Central African Republic', 'Bangui', 3),
 (3600186228, 'Syria', 'Damascus', 2),
 (3610249594, 'Bangladesh', 'Dhaka', 0),
 (3600198859, 'Comoros', 'Moroni', 0),
 (3609377730, 'São Tomé and Príncipe', 'São Tomé', 0)]

Видимо, в Ташекенте не поленились и внесли на карту все-все-все аптеки.

In [225]:
uzbekistan_area_id = 3600196240

query = overpassQueryBuilder(
    area=uzbekistan_area_id, 
    elementType=['node','way','relation'], 
    selector='"amenity"="school"',
    out='meta'
)

schools = overpass.query(query, timeout=240)

[overpass] downloading data: [timeout:240][out:json];area(3600196240)->.searchArea;(node["amenity"="school"](area.searchArea);way["amenity"="school"](area.searchArea);relation["amenity"="school"](area.searchArea);); out meta;


In [228]:
rtree_school = rtree.index.Index()
m = folium.Map()

school_distance = 0

point_afg_lon, point_afg_lat = point_afg.coords[0][0], point_afg.coords[0][1]

for i, school in enumerate(schools.toJSON()['elements']):
    if 'lon' in school and 'lat' in school:
        rtree_school.add(i, (school['lon'], school['lat'], school['lon'], school['lat']))

nearest_schools = rtree_school.nearest(
    (point_afg_lon, point_afg_lat, point_afg_lon, point_afg_lat),
    num_results=5,
)

for school_idx in nearest_schools:
    school = schools.toJSON()['elements'][school_idx]
    print('School:', school['id'], [school['lat'], school['lon']])
    folium.Marker([school['lat'], school['lon']]).add_to(m)
    school_distance += get_haversine(point_afg_lat, point_afg_lon, school['lat'], school['lon'])
    
print('Distance sum:', school_distance)
    

folium.Marker([point_afg.coords[0][1], point_afg.coords[0][0]], icon=folium.Icon(color='green')).add_to(m)  
m.location = [school['lat'], school['lon']]
m

School: 5008747121 [39.5173604, 63.839191]
School: 7193935240 [38.3728309, 66.1260062]
School: 5750722122 [37.238506, 67.2761892]
School: 5344971090 [37.4840916, 67.112056]
School: 5337519021 [38.8381183, 65.7856047]
Distance sum: 3486114.4044867205


## 2.3 Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части **

In [506]:
def extract_poly_coords(geom):
    if geom.type == 'Polygon':
        exterior_coords = geom.exterior.coords[:]
        interior_coords = []
        for interior in geom.interiors:
            interior_coords += interior.coords[:]
    elif geom.type == 'MultiPolygon':
        exterior_coords = []
        interior_coords = []
        for part in geom:
            epc = extract_poly_coords(part)  # Recursive call
            exterior_coords += epc['exterior_coords']
            interior_coords += epc['interior_coords']
    else:
        raise ValueError('Unhandled geometry type: ' + repr(geom.type))
    return {'exterior_coords': exterior_coords,
            'interior_coords': interior_coords}

In [560]:
countries_railway_bus = []

# прошу прощения за копипаст :)
for area_id, country_name in selected_countries:
    
#     print(area_id, country_name)
    
    polygon_coords_text = nominatim.query(country_name, wkt=True).toJSON()[0]['geotext']
    country_multipolygon = wkt.loads(polygon_coords_text)
    sorted_coords = sorted(extract_poly_coords(country_multipolygon)['exterior_coords'], key=lambda x: x[1])
    # самая южная и северная координаты
    (lon_south, lat_south), (lon_north, lat_north) = sorted_coords[0], sorted_coords[-1]

    query = overpassQueryBuilder(
        area=area_id, 
        elementType=['node','way','relation'], 
        selector='"railway"="station"',
        includeGeometry=True,
        out='meta'
    )
    railway_stations = overpass.query(query, timeout=240)

    query = overpassQueryBuilder(
        area=area_id, 
        elementType=['node','way','relation'], 
        selector='"highway"="bus_stop"',
        includeGeometry=True,
        out='meta'
    )
    bus_stops = overpass.query(query, timeout=540)


    railway_count = 0
    for i, el in enumerate(railway_stations.toJSON()['elements']):
        try:
            if 'lat' in el:
                lat, lon = el['lat'], el['lat']
            elif 'geometry' in el and 'lat' in el['geometry'][0]:
                lat, lon = el['geometry'][0]['lat'], el['geometry'][0]['lat']
            elif 'members' in el:
                lat, lon = el['members'][0]['geometry'][0]['lat'], el['members'][0]['geometry'][0]['lat'] 
            else:
                continue

            south_dist = get_haversine(lat_south, lon_south, lat, lon)
            north_dist = get_haversine(lat_north, lon_north, lat, lon)

            railway_count += int(south_dist < north_dist)
        except Exception as e:
            pass

    bus_stops_count = 0
    for i, el in enumerate(bus_stops.toJSON()['elements']):
        try:
            if 'lat' in el:
                lat, lon = el['lat'], el['lat']
            elif 'geometry' in el and 'lat' in el['geometry'][0]:
                lat, lon = el['geometry'][0]['lat'], el['geometry'][0]['lat']
            elif 'members' in el:
                lat, lon = el['members'][0]['geometry'][0]['lat'], el['members'][0]['geometry'][0]['lat'] 
            else:
                continue

            south_dist = get_haversine(lat_south, lon_south, lat, lon)
            north_dist = get_haversine(lat_north, lon_north, lat, lon)

            bus_stops_count += int(south_dist < north_dist)
        except Exception as e:
            pass
    
    if bus_stops_count > 0:
        countries_railway_bus.append((
            area_id,
            country_name,
            railway_count,
            bus_stops_count,
            railway_count / float(bus_stops_count)  
        ))
        print(countries_railway_bus[-1])

(3600059470, 'Brazil', 1337, 62911, 0.021252245235332454)
(3600080500, 'Australia', 1916, 48425, 0.03956633970056789)
(3600184840, 'Syria', 77, 238, 0.3235294117647059)
(3600192780, 'Liberia', 0, 4, 0.0)
(3600192790, 'Central African Republic', 2, 1, 2.0)
(3600192830, 'Cameroon', 35, 55, 0.6363636363636364)
(3600364110, 'Azerbaijan', 42, 165, 0.2545454545454545)
(3600535880, 'São Tomé and Príncipe', 0, 10, 0.0)
(3602103120, 'Brunei', 0, 64, 0.0)


In [561]:
sorted(countries_railway_bus, key=lambda x: x[4], reverse=True)

[(3600192790, 'Central African Republic', 2, 1, 2.0),
 (3600192830, 'Cameroon', 35, 55, 0.6363636363636364),
 (3600184840, 'Syria', 77, 238, 0.3235294117647059),
 (3600364110, 'Azerbaijan', 42, 165, 0.2545454545454545),
 (3600080500, 'Australia', 1916, 48425, 0.03956633970056789),
 (3600059470, 'Brazil', 1337, 62911, 0.021252245235332454),
 (3600192780, 'Liberia', 0, 4, 0.0),
 (3600535880, 'São Tomé and Príncipe', 0, 10, 0.0),
 (3602103120, 'Brunei', 0, 64, 0.0)]

В Central African Republic только 3 кинотеатра :)
Поэтому, возьмем Cameroon

In [573]:
cameroon_area_id = 3600192830

query = overpassQueryBuilder(
    area=cameroon_area_id, 
    elementType=['node','way','relation'], 
    selector='"amenity"="cinema"',
    includeGeometry=True,
    out='meta'
)
cinemas = overpass.query(query, timeout=540)

In [578]:
rtree_cinema = rtree.index.Index()
m = folium.Map()

cinema_distance = 0

point_afg_lon, point_afg_lat = point_afg.coords[0][0], point_afg.coords[0][1]

for i, cinema in enumerate(cinemas.toJSON()['elements']):
    if 'lon' in cinema and 'lat' in cinema:
        lat, lon = cinema['lat'], cinema['lon']
    elif 'geometry' in cinema:
        lat, lon = cinema['geometry'][0]['lat'], cinema['geometry'][0]['lon']
        
    rtree_cinema.add(i, (lon, lat, lon, lat))
    

nearest_cinemas = rtree_cinema.nearest(
    (point_afg_lon, point_afg_lat, point_afg_lon, point_afg_lat),
    num_results=5,
)

for cinema_idx in nearest_cinemas:
    cinema = cinemas.toJSON()['elements'][cinema_idx]
    if 'lon' in cinema and 'lat' in cinema:
        lat, lon = cinema['lat'], cinema['lon']
    elif 'geometry' in cinema:
        lat, lon = cinema['geometry'][0]['lat'], cinema['geometry'][0]['lon']
        
    print('Сinema:', cinema['id'], [lat, lon])
    folium.Marker([lat, lon]).add_to(m)
    cinema_distance += get_haversine(point_afg_lat, point_afg_lon, lat, lon)
    
print('Distance sum:', cinema_distance)
    

folium.Marker([point_afg.coords[0][1], point_afg.coords[0][0]], icon=folium.Icon(color='green')).add_to(m)  
m.location = [lat, lon]
m

Сinema: 222003170 [9.3031871, 13.3943158]
Сinema: 5916192753 [7.3206435, 13.5788742]
Сinema: 424938699 [3.8595822, 11.4963109]
Сinema: 299774304 [4.0586127, 9.7230553]
Сinema: 299702166 [4.0545266, 9.715029]
Distance sum: 30071324.608844116


In [582]:
total_distance = atm_distance + school_distance + cinema_distance
print(f'Total distance: {total_distance:.2f}m')

Total distance: 86759848.31m


## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [289]:
# put your code here

route_client = client.Client(key='5b3ce3597851110001cf6248e1f04808f25841b383ceffbc06efd199')

In [440]:
df_schools = pd.read_csv('2019_-_2020_School_Point_Locations.csv').rename(columns={'the_geom': 'geometry'})
df_schools['geometry'] = df_schools['geometry'].apply(wkt.loads)
gdf_schools = gpd.GeoDataFrame(df_schools, geometry='geometry', crs='epsg:4326')

In [441]:
point_from = [-73.96745, 40.77247]
point_to = [-73.98951, 40.70304]

map_center = [(point_from[1] + point_to[1]) / 2, (point_from[0] + point_to[0]) / 2]

In [462]:
route_area = Polygon((
    (point_from[0], point_from[1]),
    (point_to[0], point_from[1]),
    (point_to[0], point_to[1]),
    (point_from[0], point_to[1]),
)).buffer(0.03) # bbox с точками маршрута и небольшим запасом

In [466]:
school_points = gdf_schools['geometry']

# взял только школы в bbox маршрута + небольшой отступ, ибо много точек не влазит в запрос к API (HTTP ERROR 413, очень большой запрос)
school_points_near_route = school_points[school_points.within(route_area)]

school_areas = school_points_near_route.apply(
    lambda x: x.buffer(0.0005, cap_style=geometry.CAP_STYLE.round)
)
school_areas_json = school_points_near_route.to_json()
school_multipolygon = MultiPolygon(polygons=school_areas.values)

In [467]:
route_json = route_client.directions(
    coordinates=(point_from, point_to),
    format_out= 'geojson',
    geometry=True,
    format='geojson',
    instructions=False,
    options={
        'avoid_polygons': geometry.mapping(school_multipolygon),
    }
)

In [468]:
m = folium.Map(location=(map_center), zoom_start=13)
folium.map.Marker((point_from[1], point_from[0])).add_to(m)
folium.map.Marker((point_to[1], point_to[0])).add_to(m)

for point in school_points_near_route:
    folium.CircleMarker((point.coords[0][1], point.coords[0][0]), radius=1, weight=3, color='red').add_to(m)
folium.features.GeoJson(route_json).add_to(m)
folium.features.GeoJson(school_areas).add_to(m)
m